# Ejemplo de Compresión de Imágenes con SVD

Este notebook demuestra cómo usar la descomposición en valores singulares (SVD) para comprimir imágenes.

In [ ]:
import sys
import os
sys.path.insert(0, os.path.join(os.getcwd(), '..', 'src'))

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from proyecto_svd.svd_image import SVDImageProcessor

## 1. Crear una imagen de ejemplo simple

In [ ]:
# Crear una imagen de gradiente simple
width, height = 200, 200
image_array = np.zeros((height, width, 3), dtype=np.uint8)

for i in range(height):
    for j in range(width):
        image_array[i, j] = [i * 255 // height, j * 255 // width, 128]

# Guardar imagen de ejemplo
img = Image.fromarray(image_array)
example_path = '../data/ejemplo_gradiente.png'
img.save(example_path)

plt.figure(figsize=(6, 6))
plt.imshow(image_array)
plt.title('Imagen de Ejemplo')
plt.axis('off')
plt.show()

## 2. Cargar y procesar con SVD

In [ ]:
# Crear procesador y cargar imagen
processor = SVDImageProcessor(example_path)
processor.compute_svd()

print(f"Dimensiones de la imagen: {processor.image_array.shape}")
print(f"Número máximo de valores singulares: {processor.get_max_k()}")

## 3. Visualizar valores singulares

In [ ]:
# Obtener valores singulares
singular_values = processor.get_singular_values()

# Graficar
fig, axes = plt.subplots(1, 3, figsize=(15, 4))
colors = ['red', 'green', 'blue']
channels = ['Rojo', 'Verde', 'Azul']

for i, (ax, color, channel) in enumerate(zip(axes, colors, channels)):
    ax.plot(singular_values[i], color=color, linewidth=2)
    ax.set_title(f'Valores Singulares - Canal {channel}')
    ax.set_xlabel('Índice')
    ax.set_ylabel('Valor Singular')
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Comparar diferentes niveles de compresión

In [ ]:
# Probar diferentes valores de k
k_values = [5, 10, 20, 50, 100]

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

# Original
axes[0].imshow(processor.image_array)
axes[0].set_title('Original')
axes[0].axis('off')

# Comprimidas
for i, k in enumerate(k_values, 1):
    reconstructed = processor.reconstruct_image(k)
    compression_ratio = processor.get_compression_ratio(k)
    energy = processor.get_energy_retained(k)
    
    axes[i].imshow(reconstructed)
    axes[i].set_title(f'k={k}\nCompresión: {compression_ratio:.2f}x\nEnergía: {energy:.1f}%')
    axes[i].axis('off')

plt.tight_layout()
plt.show()

## 5. Análisis de energía retenida vs compresión

In [ ]:
max_k = processor.get_max_k()
k_range = range(1, min(max_k, 150), 5)

compression_ratios = []
energy_retained = []

for k in k_range:
    compression_ratios.append(processor.get_compression_ratio(k))
    energy_retained.append(processor.get_energy_retained(k))

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Energía retenida vs k
ax1.plot(k_range, energy_retained, 'b-', linewidth=2)
ax1.set_xlabel('Número de Valores Singulares (k)')
ax1.set_ylabel('Energía Retenida (%)')
ax1.set_title('Energía Retenida vs k')
ax1.grid(True, alpha=0.3)

# Ratio de compresión vs energía
ax2.plot(energy_retained, compression_ratios, 'r-', linewidth=2, marker='o', markersize=4)
ax2.set_xlabel('Energía Retenida (%)')
ax2.set_ylabel('Ratio de Compresión')
ax2.set_title('Trade-off: Compresión vs Calidad')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Ejemplo con matriz simple

Demostración matemática de SVD con una matriz pequeña:

In [ ]:
# Matriz de ejemplo
A = np.array([[3, 1, 1],
              [-1, 3, 1]], dtype=float)

print("Matriz original A:")
print(A)
print(f"\nDimensiones: {A.shape}")

# Calcular SVD
U, s, VT = np.linalg.svd(A, full_matrices=False)

print("\n" + "="*50)
print("DESCOMPOSICIÓN SVD: A = U × Σ × V^T")
print("="*50)

print("\nMatriz U (vectores singulares izquierdos):")
print(U)
print(f"Dimensiones: {U.shape}")

print("\nValores singulares (diagonal de Σ):")
print(s)
print(f"Dimensiones: {s.shape}")

print("\nMatriz V^T (vectores singulares derechos transpuestos):")
print(VT)
print(f"Dimensiones: {VT.shape}")

# Reconstruir
Sigma = np.diag(s)
A_reconstructed = U @ Sigma @ VT

print("\nMatriz reconstruida (U × Σ × V^T):")
print(A_reconstructed)

print("\nError de reconstrucción:")
print(np.max(np.abs(A - A_reconstructed)))